In [2]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime as dt

In [13]:
#cleaned = pd.read_csv('/Users/jordanlabuda/Documents/GitHub/sosc133/CleanSOSC.csv')
cleaned = pd.read_csv('CleanSOSC.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET
0,0,1003,NaN,20100131,ANTQ,0003541,2010-01-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1003,NaN,20100228,ANTQ,0003541,2010-02-28,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1003,NaN,20100331,ANTQ,0003541,2010-03-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1003,NaN,20100430,ANTQ,0003541,2010-04-30,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1003,NaN,20100531,ANTQ,0003541,2010-05-31,000354100,ANTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522
832966,832966,316056,BBB-,20161231,ALLE,G0176J1,2016-12-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.84,1601109.0,0.003931
832967,832967,316056,BBB-,20170131,ALLE,G0176J1,2017-01-31,G0176J109,ALLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
cleaned.columns

Index(['Unnamed: 0', 'gvkey', 'splticrm', 'datadate', 'tic', 'cusip_x', 'date',
       'NCUSIP', 'TICKER', 'PERMNO', 'COMNAM', 'cusip_y', 'NWPERM', 'PRC',
       'VOL', 'RET'],
      dtype='object')

Below is correct for what you wanted to do, but its a void method so we need to put cleaned= before it. Lots of pandas methods have a parameter called inplace which allows you to not have to do this.

In [16]:
#idk how to specify if only price is null but seems related so just gonna drop them all
cleaned = cleaned.dropna(subset=['PRC', 'splticrm'])

In [12]:
binned = cleaned.groupby('splticrm')

In [13]:
binned.groups

{'A': [1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1874, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, ...], 'A+': [752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 5290, 5291, 5292, 5293, 5294, 5295, 5296, 5297, 5298, 5299, 5300, 5301, 5876, 5877, 5878, 5879, 5880, 5881, 5882, 5883, 5884,

In [18]:
binsdict = {
    'D': 1,
    'SD': 2,
    'RD': 3,
    'C': 4,
    'CC': 5,
    'CCC-': 6,
    'CCC': 7,
    'CCC+': 8,
    'B-': 9,
    'B': 10,
    'B+' : 11,
    'BB-': 12,
    'BB': 13,
    'BB+': 14,
    'BBB-': 15,
    'BBB': 16,
    'BBB+' : 17,
    'A-': 18,
    'A': 19,
    'A+': 20,
    'AA-': 21,
    'AA': 22,
    'AA+': 23,
    'AAA': 24
}


In [19]:
#this is a pretty and generalizable way to apply lambdas to a dataframe
func = lambda x: binsdict[x]
cleaned['num_cr'] = cleaned['splticrm'].apply(func)

<ipython-input-19-f7a6904ae199>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['num_cr'] = cleaned['splticrm'].apply(func)


In [20]:
cleaned

,Unnamed: 0,gvkey,splticrm,datadate,tic,cusip_x,date,NCUSIP,TICKER,PERMNO,COMNAM,cusip_y,NWPERM,PRC,VOL,RET,num_cr
39,39,1004,BB,20100531,AIR,0003611,2010-05-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.82,229900.0,-0.010761,13
40,40,1004,BB,20100630,AIR,0003611,2010-06-30,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.38,184900.0,-0.034455,13
44,44,1004,BB,20101031,AIR,0003611,2010-10-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,15.36,278400.0,-0.015385,13
47,47,1004,BB,20110131,AIR,0003611,2011-01-31,000361105,AIR,54594.0,A A R CORP,00036110,NaN,24.56,308200.0,0.0107,13
48,48,1004,BB,20110228,AIR,0003611,2011-02-28,000361105,AIR,54594.0,A A R CORP,00036110,NaN,27.47,257200.0,0.004755,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832958,832958,316056,BB+,20160430,ALLE,G0176J1,2016-04-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,63.00,1127875.0,-0.013776,14
832961,832961,316056,BBB-,20160731,ALLE,G0176J1,2016-07-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,67.64,563701.0,-0.005148,15
832964,832964,316056,BBB-,20161031,ALLE,G0176J1,2016-10-31,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,71.22,573928.0,-0.000561,15
832965,832965,316056,BBB-,20161130,ALLE,G0176J1,2016-11-30,G0176J109,ALLE,14297.0,ALLEGION PLC,G0176J10,NaN,68.91,1014660.0,0.009522,15
